In [ ]:
#!pip install fastai==2.2.5

In [ ]:
from fastai.vision.all import *

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
path = Path(F"/content/gdrive/My Drive/birbcam/data/")

In [ ]:
files = get_image_files(path/"training")

In [ ]:
len(files)

In [ ]:
file_types = [str(f).split('.')[1] for f in files]
set(file_types)

In [ ]:
pattern = r'^(.*)_(\d|(\d{4})-(\d{2})-(\d{2})T(\d{2})\:(\d{2})\:(\d{2}))+.(jpg|JPG|jpeg|png)'

In [ ]:
def labeler(x):
  x = str(x)
  x = x.split('/')[-1]
  x = x.split('_')
  x = x[:-1]
  if 'none' in x:
    x.remove('none')
  return ','.join(x)

df = pd.DataFrame({'fname': [str(f).split('/')[-1] for f in files]})
df['labels'] = df['fname'].apply(labeler)
#df = df[df['labels'] != '']
df = df.reset_index(drop=true)

In [ ]:
df['labels'].unique()

In [ ]:
df.head()

In [ ]:
dls = ImageDataLoaders.from_df(df, path, folder='training', label_delim=',', 
                               item_tfms=Resize((320,240)), batch_tfms=aug_transforms(size=(320,240)))

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=partial(accuracy_multi, thresh=0.5))

In [ ]:
learn.lr_find()

In [ ]:
mixup = MixUp(1.)
learn.fine_tune(10, 3e-2, cbs=mixup)

In [ ]:
learn.save('first_cycles')

In [ ]:
learn.load('first_cycles')

In [ ]:
learn.fine_tune(5, 5e-3, cbs=mixup)

In [ ]:
learn.save('first_cycles')

In [ ]:
learn.load('first_cycles')

In [ ]:
learn.unfreeze()
learn.fine_tune(5, 5e-3, cbs=mixup)

In [ ]:
#learn.save('first_cycles')

In [ ]:
learn.fine_tune(5, 5e-3, cbs=mixup)

In [ ]:
learn.show_results()

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(16, figsize=(15,10))

In [ ]:
tst_files = get_image_files(path/"webcam")
tst_dl = dls.test_dl(tst_files)

In [ ]:
tst_dl.show_batch(max_n=8)

In [ ]:
[learn.predict(f)[0] for f in tst_files]

In [ ]:
tst_files = [t for t in files if 'chickadee_2020-12-26T09:34:22.jpg' in str(t)]
[learn.predict(f)[0] for f in tst_files]

In [ ]:
#learn.save('third_birb_model')

In [ ]:
learn.export(F"/content/gdrive/My Drive/birbcam/data/models/birbcam_prod.pkl")